In [1]:
import numpy as np
import pandas as pd

In [2]:
def sigmoid(z):
    s = 1/(1+(np.exp(-z)))
    return s

In [3]:
def initialize_with_zeros(dim):
    #initialize theta and bias to 0s
    w = np.zeros((dim, 1))
    b = 0
    return w, b

In [4]:
def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T,X)+b)
    #cost = (np.sum( (-np.log(A)*Y) + ((-np.log(1-A))*(1-Y)) ))/m
    cost = (-1/m) * ((np.dot(Y, ((np.log(A)).T))) + (np.dot((1-Y), ((np.log(1-A)).T))))
    dw = (1/m) * (np.dot(X, (A-Y).T))
    db = (1/m)* (np.sum(A-Y))
  
    grads = {"dw": dw,
             "db": db}
    cost = np.squeeze(cost)
    return grads, cost

In [5]:
def optimize(w, b, X, Y, num_itr, learningRate, print_cost = False):
    costs = []
    for i in range(num_itr):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w-(learningRate*dw)
        b = b-(learningRate*db)
        if i%100 == 0:
            costs.append(cost)
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: (%f %f %f)" %(i, cost, db, b))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [6]:
def predict(w, b, X):    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X) + b)    
    for i in range(A.shape[1]):
        if A[0, i] > 0.5:
            Y_prediction[0, i] = 1 
        else:
            Y_prediction[0, i] = 0
        
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [7]:
def model(X_train, Y_train, X_test, Y_test, num_itr = 2000, learningRate = 0.5, print_cost = False):
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_itr, learningRate, print_cost)
    w = parameters["w"]
    b = parameters["b"]
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learningRate,
         "num_iterations": num_itr}
    
    return d

For House Price Dataset

In [8]:
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('housepricedata.csv')

In [9]:
dataset

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
0,8450,7,5,856,2,1,3,8,0,548,1
1,9600,6,8,1262,2,0,3,6,1,460,1
2,11250,7,5,920,2,1,3,6,1,608,1
3,9550,7,5,756,1,0,3,7,1,642,0
4,14260,8,5,1145,2,1,4,9,1,836,1
...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,6,5,953,2,1,3,7,1,460,1
1456,13175,6,6,1542,2,0,3,7,2,500,1
1457,9042,7,9,1152,2,0,4,9,2,252,1
1458,9717,5,6,1078,1,0,2,5,0,240,0


In [10]:
X = dataset.values[:, 0:10]
Y = dataset.values[:, 10]

In [11]:
print(X)
print(Y)

[[ 8450     7     5 ...     8     0   548]
 [ 9600     6     8 ...     6     1   460]
 [11250     7     5 ...     6     1   608]
 ...
 [ 9042     7     9 ...     9     2   252]
 [ 9717     5     6 ...     5     0   240]
 [ 9937     5     6 ...     6     0   276]]
[1 1 1 ... 1 0 0]


In [12]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y, test_size=0.3)
train_set_x = X_train.T
train_set_y = Y_train.T
test_set_x = X_test.T
test_set_y = Y_test.T

In [14]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_itr = 5000, learningRate = 0.1, print_cost = False)
print("Weights: ", d["w"])
print("Bias: ", d["b"])

train accuracy: 88.5518590998043 %
test accuracy: 88.81278538812785 %
Weights:  [[ 0.61736714]
 [ 4.8793339 ]
 [-1.92354552]
 [ 1.59249583]
 [ 4.78714447]
 [ 1.69148441]
 [-1.78750156]
 [ 0.78061537]
 [ 2.85761233]
 [ 3.43495004]]
Bias:  -6.12583779968347


For AND Gate:

In [25]:
train_set_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
train_set_y = np.array([0, 0, 0, 1]).T
test_set_x = np.array([[0, 1]]).T
test_set_y = np.array([0]).T

d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_itr = 3000, learningRate = 0.01, print_cost = False)
print("Weights: ", d["w"])
print("Bias: ", d["b"])

print("\n\n Input: ", test_set_x)
print("\n\n Output: ",predict(d["w"], d["b"], test_set_x))

train accuracy: 100.0 %
test accuracy: 100.0 %
Weights:  [[1.42148221]
 [1.42148221]]
Bias:  -2.4588457124902616


 Input:  [[0]
 [1]]


 Output:  [[0.]]


For OR Gate:

In [26]:
train_set_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
train_set_y = np.array([0, 1, 1, 1]).T
test_set_x = np.array([[0, 1]]).T
test_set_y = np.array([1]).T

d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_itr = 3000, learningRate = 0.01, print_cost = False)
print("Weights: ", d["w"])
print("Bias: ", d["b"])

print("\n\n Input: ", test_set_x)
print("\n\n Output: ",predict(d["w"], d["b"], test_set_x))

train accuracy: 100.0 %
test accuracy: 100.0 %
Weights:  [[2.11923516]
 [2.11923516]]
Bias:  -0.3331368667149199


 Input:  [[0]
 [1]]


 Output:  [[1.]]


For XOR Gate:

In [28]:
train_set_x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
train_set_y = np.array([0, 1, 1, 0]).T
test_set_x = np.array([[1, 0]]).T
test_set_y = np.array([1]).T

d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_itr = 3000, learningRate = 0.001, print_cost = False)
print("Weights: ", d["w"])
print("Bias: ", d["b"])

print("\n\n Input: ", test_set_x)
print("\n\n Output: ",predict(d["w"], d["b"], test_set_x))

train accuracy: 50.0 %
test accuracy: 0.0 %
Weights:  [[0.]
 [0.]]
Bias:  0.0


 Input:  [[1]
 [0]]


 Output:  [[0.]]
